<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [8]:
yelp.shape

(10000, 9)

In [10]:
yelp.dtypes

business_id            object
cool                    int64
date           datetime64[ns]
funny                   int64
review_id              object
stars                   int64
text                   object
useful                  int64
user_id                object
dtype: object

In [2]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

In [3]:
import spacy
nlp = spacy.load("en_core_web_lg")

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
STOPWORDS = set(STOPWORDS).union(set(['yelp']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [5]:
tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

In [8]:
#this is different tokenizer method then bellow
yelp['tokens'] = yelp['text'].apply(lambda x: tokenize(x))

In [6]:
df = yelp.copy()

In [7]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [9]:
tokens = []

# Make the tokens
for doc in nlp.pipe(df['text']):
    
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop==False) & (token.is_punct==False):
           doc_tokens.append(token.text)
   
    tokens.append(doc_tokens)
    
df['tokens'] = tokens

In [10]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[BEWARE, FAKE, FAKE, FAKE, small, business, Lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[Came, lunch, Togo, Service, quick, Staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[Vegas, dozens, times, stepped, foot, Circus, ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[went, night, closed, street, party, best, act..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[3.5, 4, stars, \n\n, bad, price, $, 12.99, lu..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tf = TfidfVectorizer(stop_words = 'english')

# Create a vocabulary and get word counts per document
sparse = tf.fit_transform(df.text)

# Print word counts


# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,00,000,001695,007,00a,00am,00ish,00pm,01,011802,...,誰も乗車しなくても,豆腐花,質問にも丁寧に答えてくれましたし,車好きさんには,這是一個不錯的選擇,運転しない,運転中も英語で指導があります,食べ物はうまい,餐後點了甜點,３時間後の便
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Instantiate
from sklearn.neighbors import NearestNeighbors


# Fit on TF-IDF Vectors
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [20]:
fake_review = [""" Wow this is absolute crap I cannot believe I wasted
my money on this, if I don't get a full refund someone is going to get sued
someone should literally be fired for this. you are shit."""]

In [21]:
# Query for Sim of Random doc to BBC
new = tf.transform(fake_review)

nn.kneighbors(new.todense())

(array([[1.28828901, 1.30107269, 1.30255805, 1.30263585, 1.30651596,
         1.31174753, 1.31279196, 1.31291271, 1.32646547, 1.32654386]]),
 array([[6847, 3330, 5260, 5081, 6027, 9163, 1416, 8290,  964, 1021]]))

In [22]:
df['text'][6847]

'Just because I was in the area and I saw the show, worst experience and crap food.'

In [23]:
df['text'][3330]

"WOW! I should have read these reviews before walking in to this Little Caesars. Appearantly the people running this store don't give a crap about the nasty customer service at this location.  They lost a customer for sure. Never again"

In [24]:
#this one is not relevant
df['text'][5260]

'I had the pizza and it was brick fired amazing! I look forward to going back and exploring more from the menu. Several great options and the burgers look delicious.'

In [25]:
df['text'][5081]

"Consider me another sucker!!  It's so rare to find ANY business nowadays that deals in new, unopened merchandise that refuses to refund their own products that you kinda forget to be careful and see the sign that's there that ALL SALES FINAL NO REFUNDS.   I guess even they don't want their own stuff back.  Lesson learned, glad i didn't spend too much money.  Will definitely pay MORE MONEY if necessary to shop at a reputable business like Macy's or Amazon or even Walmart for that matter that will happily refund your money for unopened returns of their own product.  Some times exchanges are fine but when it's something that you really want to return and get a refund don't go here.  Unfortunately changing your mind is not allowed.  After dealing with this runaround I returned a pair of sunglasses that I actually wore for a few hours and didn't like at a different store with ABSOLUTELY NO HASSLE OF ANY KIND.  Customer service is more than being courteous to customers.... if I could give z

In [26]:
df['text'][1021]

'How do they stay in business? Smelled like crap. Order took forever and the place was empty. Terrible atmosphere and food.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [27]:
# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

In [28]:
# Create Pipeline

vect = TfidfVectorizer(stop_words='english')
sgdc = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [30]:
# Fit Pipeline
pipe.fit(df.text, df.stars)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...imators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [31]:
# 1 star makes absolute sense!
fake_star = pipe.predict(fake_review)
fake_star

array([1])

In [38]:
# Experiment Management
from sklearn.model_selection import GridSearchCV

parameters = {
    
}

In [39]:
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)

In [40]:
grid_search.fit(df.text, df.stars)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...imators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [43]:
fake_stars2 = grid_search.predict(fake_review)
fake_stars2

array([1])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [42]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [43]:
id2word = corpora.Dictionary(yelp['tokens'])

In [44]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(24, 1), (781, 1), (10164, 3)]

In [24]:
import sys
sys.getsizeof(id2word)

56

In [29]:
len(id2word.keys())

26474

In [45]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=10, no_above=0.75)

In [50]:
len(id2word.keys())

4781

In [51]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 5,
                   passes=10,
                   workers=4
                  )

In [49]:
lda.print_topics()

[(0,
  '0.015*"place" + 0.010*"love" + 0.010*"coffee" + 0.010*"great" + 0.008*"like" + 0.007*"store" + 0.007*"time" + 0.006*"new" + 0.005*"location" + 0.005*"shop"'),
 (1,
  '0.015*"food" + 0.011*"good" + 0.011*"place" + 0.010*"ordered" + 0.009*"like" + 0.008*"sushi" + 0.007*"time" + 0.007*"service" + 0.007*"great" + 0.007*"got"'),
 (2,
  '0.011*"time" + 0.010*"service" + 0.007*"great" + 0.007*"like" + 0.007*"place" + 0.006*"said" + 0.006*"got" + 0.005*"told" + 0.005*"people" + 0.005*"staff"'),
 (3,
  '0.031*"food" + 0.019*"place" + 0.018*"good" + 0.016*"great" + 0.013*"service" + 0.009*"best" + 0.008*"like" + 0.008*"chicken" + 0.007*"delicious" + 0.007*"time"'),
 (4,
  '0.018*"good" + 0.011*"great" + 0.011*"food" + 0.010*"like" + 0.009*"bar" + 0.008*"place" + 0.006*"nice" + 0.006*"time" + 0.005*"menu" + 0.005*"service"')]

In [52]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [53]:
topics = [' '.join(t[0:5]) for t in words]

In [54]:
print(topics[0])

place love coffee great like


In [55]:
for t in topics:
    print(t)
    print("\n")

place love coffee great like


food good place ordered like


time service great like place


food place good great service


good great food like bar




In [56]:
import warnings
warnings.filterwarnings('ignore')

In [57]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [58]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.158333  0.065426       1        1  33.094109
3      0.088370 -0.010409       2        1  21.112541
4      0.070899 -0.000979       3        1  18.672586
1      0.064254  0.060455       4        1  16.325687
0     -0.065190 -0.114493       5        1  10.795078, topic_info=     Category         Freq        Term        Total  loglift  logprob
133   Default  4857.000000        food  4857.000000  30.0000  30.0000
37    Default  4557.000000        good  4557.000000  29.0000  29.0000
122   Default  4715.000000       place  4715.000000  28.0000  28.0000
991   Default   639.000000      coffee   639.000000  27.0000  27.0000
682   Default   536.000000       sushi   536.000000  26.0000  26.0000
1038  Default  1380.000000     chicken  1380.000000  25.0000  25.0000
702   Default   714.000000       store   714.000000  24.0000  24.0000
638   Default  1643.000000        love  1643.000000  23.0000  23.0000
126   Default  1060.000000         bar  1060.000000  22.0000  22.0000
1028  Default  1540.000000     ordered  1540.000000  21.0000  21.0000
348   Default   780.000000       pizza   780.000000  20.0000  20.0000
570   Default  1341.000000  restaurant  1341.000000  19.0000  19.0000
904   Default   501.000000        beer   501.000000  18.0000  18.0000
318   Default  1228.000000   delicious  1228.000000  17.0000  17.0000
38    Default   753.000000       lunch   753.000000  16.0000  16.0000
165   Default  1218.000000        menu  1218.000000  15.0000  15.0000
309   Default   827.000000      cheese   827.000000  14.0000  14.0000
188   Default  4320.000000       great  4320.000000  13.0000  13.0000
1666  Default   640.000000         car   640.000000  12.0000  12.0000
266   Default   422.000000        shop   422.000000  11.0000  11.0000
1735  Default   536.000000      burger   536.000000  10.0000  10.0000
676   Default   540.000000        rice   540.000000   9.0000   9.0000
2010  Default   339.000000       tacos   339.000000   8.0000   8.0000
216   Default   576.000000        meat   576.000000   7.0000   7.0000
328   Default   930.000000       fresh   930.000000   6.0000   6.0000
353   Default   803.000000       sauce   803.000000   5.0000   5.0000
482   Default   742.000000        meal   742.000000   4.0000   4.0000
881   Default  1178.000000         new  1178.000000   3.0000   3.0000
1245  Default   589.000000       fries   589.000000   2.0000   2.0000
401   Default   847.000000         eat   847.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
638    Topic5   456.716248        love  1643.599121   0.9455  -4.6125
2232   Topic5   142.818817       local   324.198395   1.4063  -5.7749
687    Topic5    80.156158        cafe   141.320572   1.6590  -6.3525
185    Topic5   163.450302         fun   454.831116   1.2027  -5.6400
122    Topic5   682.255859       place  4715.338379   0.2929  -4.2111
881    Topic5   281.597961         new  1178.006104   0.7950  -5.0960
937    Topic5   226.626709    location   891.876221   0.8561  -5.3132
890    Topic5    80.927002       seats   152.638031   1.5916  -6.3430
75     Topic5   129.001495        kids   368.246460   1.1772  -5.8767
188    Topic5   437.585297       great  4320.976074  -0.0639  -4.6552
15     Topic5   359.727264        like  3440.491699  -0.0319  -4.8512
348    Topic5   168.170975       pizza   780.031799   0.6917  -5.6115
630    Topic5   150.024155       drink   614.197083   0.8166  -5.7257
31     Topic5   314.781616        time  3475.860352  -0.1756  -4.9846
43     Topic5   212.876923       staff  1572.879150   0.2261  -5.3758
841    Topic5   151.141541     looking   713.462646   0.6742  -5.7183
473    Topic5   169.185608        work   961.486206   0.4886  -5.6055
111    Topic5   214.615616        best  1870.216675   0.0611  -5.3677
46     Topic5   149.491318        area   753.813965   0.6082  -5.7293
176    To

In [ ]:
### hmmm i a pretty thrown that all the topics are positive seeming and seem so close related...
# i was almost expecting topics to be broken up by star 
# that there would be one super positive, one super negative, and some middle ones... hmmmm
# it is very possible i did not make enough topics and most reviews are more postitive then not... 
# i guess i could check that with star distribution

## wow wrote that top piece while visualiztion was loading... now that it is loaded i am pretty impressed by it
# the topics were not associated with the star rating of the review but rather what was being reviewed 
# and it seems to have done a pretty awesome job of splitting the topics.
# for the five it seems like there is bar, bar-type food, asian food, coffee shop, and retail store
#with bar and bar food are overlapped


## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)